In [1]:
#### reloads modules automatically - GREAT INCLUSION
using Revise

### !! IMPORTANT!! NEED THIS LINE TO FIND CUSTOM MODULES
!(pwd() in LOAD_PATH) ? push!(LOAD_PATH, pwd()) : nothing

using BenchmarkTools
using CSV
using DataFrames
using Distributed


#(nprocs() == 1) && addprocs(10)

@everywhere using DistributedArrays
@everywhere using Graphs
@everywhere using LinearAlgebra
using Plots
using Random
using SharedArrays
@everywhere using SparseArrays
using Suppressor



# load IterativeHeaps
#using Pkg
#Pkg.develop(path = "/Users/jsyme/Documents/Projects/git_jbus/IterativeHeaps.jl")
using IterativeHeaps

# load GraphDistanceAlgorithms
#Pkg.develop(path = "/Users/jsyme/Documents/Projects/git_jbus/GraphDistanceAlgorithms.jl")
using GraphDistanceAlgorithms

# get the graph fragments package
#Pkg.develop(path = "/Users/jsyme/Documents/Projects/git_jbus/GraphFragments.jl/")
using GraphFragments

# get the DiscreteGraphAlgorithms package
#Pkg.develop(path = "/Users/jsyme/Documents/Projects/git_jbus/DiscreteGraphAlgorithms.jl/")
using DiscreteGraphAlgorithms

In [62]:
# fp_krebs_out = "/Users/jsyme/Documents/Projects/FY23/KPP_RIR/edge_lists/krebs_test.egl"
# graph_wrapper_krebs = read_egl(fp_krebs_out);

fp_bell = "./bell_example_fig_10.csv"
@time graph_wrapper_bell = read_egl(fp_bell; force_undirected = true);


  0.000591 seconds (662 allocations: 97.039 KiB)


In [482]:
dir_read = "/Users/jsyme/Documents/Projects/FY23/KPP_RIR/edge_lists"

# get graph wrappers
files = [x for x in readdir(dir_read) if endswith(x, "tsv")]
dict_gws = Dict{Symbol, GraphWrapper}()

for file in files[1:1]
    fp = joinpath(dir_read, file)
    key = Symbol(replace(file, " hop(s).tsv" => "", " " => "_"))
    
    dict_gws[key] = read_egl(
        fp;
        delim = "\t",
        force_undirected = true,
        infer_weights = false,
    )
    
    @info "Finished reading graph `$(key)`"
end


[ Info: Finished reading graph `finance_2`


In [150]:
#dict_gws[:finance_2]

Graphs.DijkstraState{Int64, Int64}([0, 3, 17, 2, 20, 5, 5, 5, 5, 4  …  16, 5, 1, 14, 17, 14, 5, 17, 2, 12], [0, 4, 3, 5, 6, 7, 7, 7, 7, 6  …  4, 7, 1, 2, 3, 2, 7, 3, 5, 5], [Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[]  …  Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[], Int64[]], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0  …  1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Int64[])

50-element Vector{String}:
 "(Korea) Daebong Shipping Company (2003)"
 "222 Shipping & Trading (2007)"
 "5th Machine Industry Bureau"
 "AMM Middle East General Trading"
 "AUSTRIA"
 "Abdul Qadeer Khan"
 "Above Energy Pty Ltd"
 "Above Engineering Pty Ltd"
 "Academy of National Defense Science"
 "Adnet International"
 "Adra missile base"
 "Advance Auto"
 "Advance KY Corporation"
 ⋮
 "BANGLADESH"
 "BARBADOS"
 "BELARUS"
 "BOLIVIA"
 "BOTSWANA"
 "BRAZIL"
 "BRITISH VIRGIN ISLANDS"
 "Baek Ma Kang"
 "Baek Sa bong"
 "Bai Mei 8"
 "Baili Shipping and Trading Limited"
 "Banco Commerciale Lugano"

In [161]:
# indices of groups that we're interested in 
gw = dict_gws[:finance_2]#graph_wrapper_bell
group = gw.vertex_names[1:50]#string.([5, 6, 8, 9, 10, 11, 12, 13])




inds_group = findall(x -> (x in group), gw.vertex_names);


# get the source (can iterate over V_S)
vertex = 1
@time state = dijkstra_shortest_paths(gw.graph, vertex; allpaths = true, )

# vector of numerators and denominators
vec_nums = zeros(Float64, gw.dims[1])
vec_denom = length.(state.predecessors)
vec_onpath = zeros(Int64, gw.dims[1])

# iterate over each other vertex - j is the target vertex, pred are the predecessors to that vertex; at j == i, length(pred) == 0
for (j, pred) in enumerate(state.predecessors)
    
    # iterate over predecessors
    for k in pred
        
        it = 1 
       
    end
end



  0.000644 seconds (1.90 k allocations: 291.484 KiB)


In [427]:
#addprocs(4)
graph_wrapper_bell.dims

(21, 56)

In [253]:
?remotecall_fetch

search: remotecall_fetch remotecall_wait



```
remotecall_fetch(f, id::Integer, args...; kwargs...)
```

Perform `fetch(remotecall(...))` in one message. Keyword arguments, if any, are passed through to `f`. Any remote exceptions are captured in a [`RemoteException`](@ref) and thrown.

See also [`fetch`](@ref) and [`remotecall`](@ref).

# Examples

```julia-repl
$ julia -p 2

julia> remotecall_fetch(sqrt, 2, 4)
2.0

julia> remotecall_fetch(sqrt, 2, -4)
ERROR: On worker 2:
DomainError with -4.0:
sqrt was called with a negative real argument but will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).
...
```

---

```
remotecall_fetch(f, pool::AbstractWorkerPool, args...; kwargs...) -> result
```

[`WorkerPool`](@ref) variant of `remotecall_fetch(f, pid, ....)`. Waits for and takes a free worker from `pool` and performs a `remotecall_fetch` on it.

# Examples

```julia-repl
$ julia -p 3

julia> wp = WorkerPool([2, 3]);

julia> A = rand(3000);

julia> remotecall_fetch(maximum, wp, A)
0.9995177101692958
```


In [255]:
@everywhere dict_path = Dict()

In [281]:
nprocs()

In [277]:
remotecall_fetch(
    () -> (push!(dict_path, "sum_$(myid())" => 2)),
    2
)

Dict{Any, Any} with 2 entries:
  "test"  => 2
  "sum_2" => 2

In [483]:
"""
Recursive function to retrieve # of paths + number of paths on which each 
    vertex on the tree sits on the path (for betweenness centrality)
"""
function get_path_counts(
    states::Vector, 
    node::Vector,
    dict_paths::Dict,
    dict_total::Dict,
    parent::Vector{Int64},
    parents::Vector,
)
    # here is where you want to check the node's name against your list of
    # 'nodes to shout out about'
    
    (length(node) == 0) && (return 1)
    
    
    for (i, child) in enumerate(node)
        
        #push!(parents, parent...)
        parents = parent
        
        # add to paths
        if child in keys(dict_paths)
            dict_paths[child] += 1
        else
            dict_paths[child] = max(length(states[child]), 1)
        end

        
        #print("i = $(i)\nchild = $(child)\nnext = $(states[child])\n")
        val = get_path_counts(
                states, 
                states[child], 
                dict_paths, 
                dict_total, 
                [parent; [child]],
                parents,
        )
            
        dict_total["sum"] += val
        
        (val == 1) && empty!(parents) # println(parents);
        
        #(val == 1) && (empty!(parents))
        #println("$(i) done.\n\n")
    end
    
    return 0
end



"""

# Constructs

get_paths(
    states::Vector, 
    node::Vector,
    parent::Vector{Int64},
    parents::Vector,
    paths::Vector{Int64},
)


##  Function Arguments


##  Keyword Arguments

"""
function get_paths(
    states::Vector{Vector{Int64}}, 
    node::Vector{Int64},
    parent::Vector{Int64},
    parents::Vector{Int64},
    paths::Vector{Vector{Int64}},
)
    # here is where you want to check the node's name against your list of
    # 'nodes to shout out about'
    
    (length(node) == 0) && (return 1)
    
    
    for (i, child) in enumerate(node)
        
        #push!(parents, parent...)
        parents = parent
        
        #print("i = $(i)\nchild = $(child)\nnext = $(states[child])\n")
        val = get_paths(
                states, 
                states[child], 
                [parent; [child]],
                parents,
                paths,
        )
            

        (val == 1) && (push!(paths, parents[2:end]); empty!(parents))

    end
    
    return 0
end



"""
Using a vector of vectors of ordered paths, calculate the number of times
    each vertex is on a path (keys in output dictionary)
"""
function get_path_counts_by_vertex(
    vec_paths::Vector,
)
    dict_out = Dict()
    
    # iter
    for (i, vec) in enumerate(vec_paths)

        for el in vec
            (el in keys(dict_out)) ? (dict_out[el] += 1) : (dict_out[el] = 1)
        end
    end
    
    return dict_out
end




"""
Calculate the component betweenness centrality for paths starting at i and heading
    to all other vertices
"""
function b_i!(
    graph::AbstractGraph, # graph_wrapper_bell.graph
    vertex::Int64,
    vec_component::Vector{Float64};
    vertices_target::Union{Vector{Int64}, Nothing} = nothing,
)
    
    # initialize components
    
    # calculate shortest paths
    state = dijkstra_shortest_paths(
        graph, 
        vertex;
        allpaths = true,
    )
    
    vertices_target = isa(vertices_target, Nothing) ? collect(vertices(graph)) : vertices_target
    
    # iterate over predecessors 
    for j in vertices_target
        
        preds = state.predecessors[j]
        
        # skip if there is no path
        (length(preds) == 0) && continue
        
        vec_paths = Vector{Vector{Int64}}()
        vec_parents = Vector{Int64}()
        
        # get all paths
        paths_all = get_paths(
            state.predecessors, 
            preds,
            [j],
            vec_parents,
            vec_paths,
        )
        
        dict_counts = get_path_counts_by_vertex(vec_paths, )
        
        for (k, v) in dict_counts
            vec_component[k] += v/state.pathcounts[j]
        end
    end
    
    return nothing
end
    



"""
Slow implementation of betweenness centrality that can be used to implment Bell's centrality.

Specify groups using vertices_source and vertices_target
"""
function betweenness_centrality_bells(
    graph::AbstractGraph;
    vertices_source::Union{Vector{Int64}, Nothing} = nothing,
    vertices_target::Union{Vector{Int64}, Nothing} = nothing,
)
    # get 
    vertices_source = isa(vertices_source, Nothing) ? collect(vertices(graph)) : vertices_source
    vertices_target = isa(vertices_target, Nothing) ? collect(vertices(graph)) : vertices_target
    vec_betweenness_centrality = zeros(Float64, nv(graph))

    for i in vertices_source
        b_i!(
            graph,
            i,
            vec_betweenness_centrality;
            vertices_target = vertices_target,
        )
    end

    !is_directed(graph) && (vec_betweenness_centrality ./= 2)
    
    return vec_betweenness_centrality
end





betweenness_centrality_bells (generic function with 1 method)

In [ ]:
v = dijkstra_shortest_paths(dict_gws[:finance_2].graph, 195; allpaths = true).predecessors

In [493]:
inds_out = sort(setdiff(vertices(dict_gws[:finance_2].graph), inds_group))

1490-element Vector{Int64}:
   51
   52
   53
   54
   55
   56
   57
   58
   59
   60
   61
   62
   63
    ⋮
 1529
 1530
 1531
 1532
 1533
 1534
 1535
 1536
 1537
 1538
 1539
 1540

In [499]:
@time vertex_in = betweenness_centrality_bells(
    dict_gws[:finance_2].graph,
    vertices_source = inds_group,
    vertices_target = inds_group,
);

  0.044425 seconds (269.36 k allocations: 23.716 MiB)


In [494]:
@time vertex_out = betweenness_centrality_bells(
    dict_gws[:finance_2].graph,
    vertices_source = inds_out,
    vertices_target = inds_out,
)

  8.522727 seconds (157.79 M allocations: 8.409 GiB, 7.63% gc time)


1540-element Vector{Float64}:
    0.0
    0.0
    0.0
  636.3752200173503
  123.56609276460571
    0.0
    0.0
    0.0
    0.0
    0.0
 2050.3050259686875
    0.0
    0.0
    ⋮
  780.3453857653029
  854.6854521932022
    0.5
  412.00848310784113
    0.0
  571.0994473829887
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0

In [503]:
@time vertex_boundary = betweenness_centrality_bells(
    dict_gws[:finance_2].graph,
    vertices_source = inds_group,
    vertices_target = inds_out,
);

  0.296653 seconds (5.31 M allocations: 289.989 MiB, 7.32% gc time)


In [502]:
@time vertex_boundary2 = betweenness_centrality_bells(
    dict_gws[:finance_2].graph,
    vertices_source = inds_out,
    vertices_target = inds_group,
)

  1.119576 seconds (7.98 M allocations: 701.966 MiB, 5.33% gc time)


1540-element Vector{Float64}:
   0.0
   0.0
   0.0
  47.382339243391
   1.932271432271431
   0.0
   0.0
   0.0
   0.0
   0.0
  35.05944366727536
   0.0
   0.0
   ⋮
  16.142511859149796
 366.26323102217486
   0.0
   5.991517605488194
   0.0
   9.818660287081338
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [506]:
maximum(abs.(vertex_in + vertex_out + vertex_boundary + vertex_boundary2 - betweenness_centrality(dict_gws[:finance_2].graph; normalize = false)))

1.360895112156868e-7

In [ ]:
@time vertex_out = betweenness_centrality_bells(
    dict_gws[:finance_2].graph,
    vertices_source = inds_out,
    vertices_target = inds_out,
)

In [487]:
@time betweenness_centrality(dict_gws[:finance_2].graph; normalize = false)

  0.969436 seconds (2.93 M allocations: 483.011 MiB, 5.95% gc time)


1540-element Vector{Float64}:
    0.0
    0.0
    0.0
  732.8545538293175
  127.43063562914858
    0.0
    0.0
    0.0
    0.0
    0.0
 2120.4239133030296
    0.0
    0.0
    ⋮
  812.630409483601
 1610.603312653981
    0.5
  423.99151831881505
    0.0
  590.73676795715
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0

1-element Vector{String}:
 "is_directed"

In [420]:
paths
vec_p

Int64[]

In [359]:
@time get_path_counts(
    state.predecessors, 
    state.predecessors[356],
    path_counts,
    dict_total_paths,
    [356],
    vec_p
);


[356, 1372, 843, 195]
[356, 437, 843, 195]
[356, 718, 962, 195]
[356, 718, 68, 195]
[356, 1220, 261, 195]
[356, 673, 778, 195]
[356, 673, 1252, 195]
[356, 673, 658, 195]
[356, 358, 261, 195]
[356, 1425, 1222, 195]
[356, 325, 93, 195]
[356, 360, 364, 195]
  0.006592 seconds (1.63 k allocations: 42.594 KiB)


In [305]:
# p1: 356 -> 1372 -> 843 -> 195 -> 1
# p2: 356 -> 437 -> 843 -> 195 -> 1
# p3: 356 -> 718 -> 962 -> 195 -> 1
# p4: 356 -> 718 -> 68 -> 195 -> 1
# p5: 356 -> 1220 -> 261 -> 195 -> 1
# p6: 356 -> 673 -> 778 -> 195 -> 1
# p7: 356 -> 673 -> 1252 -> 195 -> 1
# p8: 356 -> 673 -> 658 -> 195 -> 1
# p9: 356 -> 358 -> 261 -> 195 -> 1
# p10: 356 -> 1425 -> 1222 -> 195 -> 1
# p11: 356 -> 325 -> 93 -> 195 -> 1
# p12: 356 -> 360 -> 364 -> 195 -> 1
path_counts

#dict_total_paths




Dict{Any, Any} with 21 entries:
  360  => 1
  843  => 2
  962  => 1
  195  => 12
  778  => 1
  325  => 1
  1    => 12
  718  => 1
  1425 => 1
  1222 => 1
  68   => 1
  673  => 1
  1252 => 1
  1372 => 1
  93   => 1
  364  => 1
  261  => 2
  358  => 1
  1220 => 1
  437  => 1
  658  => 1

In [286]:
#findall(x -> (x == 9), length.(state.predecessors))#[1530]
a = state.predecessors[673]
a


3-element Vector{Int64}:
  778
 1252
  658

In [240]:
state.predecessors[a[6]]

1-element Vector{Int64}:
 261

In [239]:
state.predecessors[261]

1-element Vector{Int64}:
 195

In [188]:
state.predecessors[93]

1-element Vector{Int64}:
 195

In [178]:
state.predecessors[195]

1-element Vector{Int64}:
 1

In [142]:
@time walk_tree(
    state.predecessors, 
    state.predecessors[7], 
    paths,
    total,
)

  0.000046 seconds


0

In [85]:
function this_one(vec::Vector, i0::Int64)
    v = []
    out = vec[i0]
    push!(v, this_one[out])
    
    return v
end


this_one(state.predecessors, 1)

LoadError: MethodError: no method matching getindex(::typeof(this_one), ::Vector{Int64})

In [24]:
graph_wrapper_bell.vertex_names

21-element Vector{String}:
 "1"
 "10"
 "11"
 "12"
 "13"
 "14"
 "15"
 "16"
 "17"
 "18"
 "19"
 "2"
 "20"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 "9"
 "\ufeff1"